## The following dataset includes taxi trips from 2013 to the present, reported to the City of Chicago in its role as a regulatory agency. It is publicly available to anyone to use for analysis.
#### https://cloud.google.com/bigquery/public-data/chicago-taxi

### Most installations below need to be applied only once, and is specific if you are looking to work with BigQuery Datasets

In [ ]:
#pip install --upgrade google-api-python-client
#pip install google-cloud-bigquer
#pip install pyarrow
#!pip install --upgrade shapely

#### Necessary imports

In [52]:
from google.cloud import bigquery

#### The JSON file containing application credentials is unique to each user and must be set up and authenticated using Google Cloud API

##### https://cloud.google.com/docs/authentication/getting-started

In [2]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/akshayaparthasarathy/Desktop/WORK/KaggleNotebooks/chicago-taxi-trips-332312-e3dcbda772ab.json"
import pandas as pd

### Setting client & dataset reference and fetching the required tables.

In [3]:
client = bigquery.Client()
dataset_ref = client.dataset("chicago_taxi_trips", 
                              project="bigquery-public-data")
dataset = client.get_dataset(dataset_ref)

In [4]:
tables = list(client.list_tables(dataset))
#for table in tables:
    #print(table.table_id)

In [5]:
table_ref = dataset_ref.table("taxi_trips")
table = client.get_table(table_ref)

In [6]:
table.schema

[SchemaField('unique_key', 'STRING', 'REQUIRED', 'Unique identifier for the trip.', (), None),
 SchemaField('taxi_id', 'STRING', 'REQUIRED', 'A unique identifier for the taxi.', (), None),
 SchemaField('trip_start_timestamp', 'TIMESTAMP', 'NULLABLE', 'When the trip started, rounded to the nearest 15 minutes.', (), None),
 SchemaField('trip_end_timestamp', 'TIMESTAMP', 'NULLABLE', 'When the trip ended, rounded to the nearest 15 minutes.', (), None),
 SchemaField('trip_seconds', 'INTEGER', 'NULLABLE', 'Time of the trip in seconds.', (), None),
 SchemaField('trip_miles', 'FLOAT', 'NULLABLE', 'Distance of the trip in miles.', (), None),
 SchemaField('pickup_census_tract', 'INTEGER', 'NULLABLE', 'The Census Tract where the trip began. For privacy, this Census Tract is not shown for some trips.', (), None),
 SchemaField('dropoff_census_tract', 'INTEGER', 'NULLABLE', 'The Census Tract where the trip ended. For privacy, this Census Tract is not shown for some trips.', (), None),
 SchemaField('

In [57]:
column_names_query = """

SELECT column_name
FROM `bigquery-public-data.chicago_taxi_trips.INFORMATION_SCHEMA.COLUMNS`
WHERE table_name = 'taxi_trips'


"""

In [58]:
query_job = client.query(column_names_query)
query_result = query_job.to_dataframe()
print(query_result)

               column_name
0               unique_key
1                  taxi_id
2     trip_start_timestamp
3       trip_end_timestamp
4             trip_seconds
5               trip_miles
6      pickup_census_tract
7     dropoff_census_tract
8    pickup_community_area
9   dropoff_community_area
10                    fare
11                    tips
12                   tolls
13                  extras
14              trip_total
15            payment_type
16                 company
17         pickup_latitude
18        pickup_longitude
19         pickup_location
20        dropoff_latitude
21       dropoff_longitude
22        dropoff_location


#### Quick view of the dataset columns

In [7]:
client.list_rows(table, max_results=5).to_dataframe()

unique_key  \
0  e5ab129c155c3719216beb5dcbf2aa8ca19e5656   
1  a5857e21a97c70612af9b1045b8bd7a626c558ef   
2  2f5f796136c7e11e48d3bb08c8c6d73dfbe0e88c   
3  03cf9fabe105982c7b0382c54e002e43dd739c3a   
4  1ba3648418a92ae0743abf80ca631cca51446d5b   

                                             taxi_id  \
0  c1aac0b2efe1a131a40f9761b4f5c9dcad9168b29cac42...   
1  c6e0071dc1db129271f6d4833953bb62bd466a9aed3330...   
2  54044c4ce47d539663e1520272de87e38a14d38ca23a66...   
3  2ee325e10e1eec919fc5bfc4d64ed9951cbab4fb13b3e2...   
4  16a233f62883c48f7462a0d5b87191190c49a46fe52f37...   

       trip_start_timestamp        trip_end_timestamp  trip_seconds  \
0 2014-07-30 23:00:00+00:00 2014-07-30 23:00:00+00:00             0   
1 2014-07-30 09:30:00+00:00 2014-07-30 09:30:00+00:00             0   
2 2014-07-13 07:00:00+00:00 2014-07-13 07:00:00+00:00             0   
3 2014-07-07 23:00:00+00:00 2014-07-07 23:00:00+00:00             0   
4 2014-08-28 16:45:00+00:00 2014-08-28 16:45:00+00:00             0   

   trip_miles  pickup_census_tract  dropoff_census_tract  \
0         0.0                  NaN                   NaN   
1         0.0                  NaN                   NaN   
2         0.0                  NaN                   NaN   
3         0.0                  NaN                   NaN   
4         0.0                  NaN                   NaN   

   pickup_community_area  dropoff_community_area  ...  extras  trip_total  \
0                    NaN                     NaN  ...     0.0       10.62   
1                    NaN                     NaN  ...     0.0       12.25   
2                    NaN                     NaN  ...     0.0       41.10   
3                    NaN                     NaN  ...     0.0        5.56   
4                    NaN                     NaN  ...     0.0        6.85   

   payment_type                                   company  pickup_latitude  \
0   Credit Card  Chicago Elite Cab Corp. (Chicago Carriag              NaN   
1   Credit Card                     T.A.S. - Payment Only              NaN   
2   Credit Card  Chicago Elite Cab Corp. (Chicago Carriag              NaN   
3   Credit Card  Chicago Elite Cab Corp. (Chicago Carriag              NaN   
4   Credit Card                     T.A.S. - Payment Only              NaN   

  pickup_longitude pickup_location  dropoff_latitude  dropoff_longitude  \
0              NaN            None               NaN                NaN   
1              NaN            None               NaN                NaN   
2              NaN            None               NaN                NaN   
3              NaN            None               NaN                NaN   
4              NaN            None               NaN                NaN   

  dropoff_location  
0             None  
1             None  
2             None  
3             None  
4             None  

[5 rows x 23 columns]

#### Setting up Query Job from client to run SQL queries. Additonally setting up configuration to cancel the query if it would use too much of your quota, with the limit set to 10 GB.

In [11]:
sample_query = """

SELECT DISTINCT company FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips` 
LIMIT 10

"""

In [12]:
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(sample_query, job_config=safe_config)

In [13]:
query_result = query_job.to_dataframe()
print(query_result)

                          company
0         5997 - AW Services Inc.
1            2241 - Manuel Alonso
2               2733 - Benny Jona
3          3319 - C&D Cab Company
4  4523 - 79481 Hazel Transit Inc
5        4787 - 56058 Reny Cab Co
6     0118 - 42111 Godfrey S.Awir
7                6488 - Zuha Taxi
8                      Globe Taxi
9      1408 - 89599 Donald Barnes


### Following questions were a part of the Kaggle notebook queries.

#### Q1: Taxi company usually used in chicago

In [14]:

popular_company = """

SELECT DISTINCT company, count(*) AS Frequency_Of_Use FROM
`bigquery-public-data.chicago_taxi_trips.taxi_trips`
GROUP BY company
ORDER BY Frequency_Of_Use DESC
LIMIT 3

"""

In [15]:
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(popular_company, job_config=safe_config)
popular_comp_result = query_job.to_dataframe()
print(popular_comp_result)

                     company  Frequency_Of_Use
0  Taxi Affiliation Services          41396001
1                       None          33593407
2                  Flash Cab          18620682


#### Q2: Most popular mode of Payment used

In [16]:
frequent_payment_type = """

SELECT distinct payment_type, COUNT(1) AS no_of_uses FROM
`bigquery-public-data.chicago_taxi_trips.taxi_trips`
GROUP BY payment_type
ORDER BY no_of_uses DESC

"""

In [17]:
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(frequent_payment_type, job_config=safe_config)
payment_result = query_job.to_dataframe()
print(payment_result)

   payment_type  no_of_uses
0          Cash   114508928
1   Credit Card    80108037
2       Unknown      868647
3        Prcard      861286
4     No Charge      817082
5        Mobile      628237
6       Dispute       82851
7         Pcard       36874
8         Split        3442
9       Prepaid        1805
10     Way2ride         142


#### Q3: What are the maximum, minimum and average fares for rides lasting 10 minutes or more?

In [47]:
fare_calc = """

WITH FARE_CALC AS(

SELECT 
RIGHT(taxi_id, 5) AS taxi_id,
ROUND(MAX(fare), 2) AS max_fare,
ROUND(MIN(fare), 2) AS min_fare,
ROUND(AVG(fare), 2) AS avg_fare,
EXTRACT(MINUTE FROM trip_start_timestamp) AS start_time,
EXTRACT(MINUTE FROM trip_end_timestamp) AS end_time
FROM 
`bigquery-public-data.chicago_taxi_trips.taxi_trips`
GROUP BY taxi_id, start_time, end_time
)

SELECT taxi_id,
max_fare,
min_fare,
avg_fare,
end_time - start_time AS trip_duration
FROM FARE_CALC 
WHERE end_time - start_time >= 10


"""


In [50]:
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(fare_calc, job_config=safe_config)
fare_result = query_job.to_dataframe()
print(fare_result.head(10))

  taxi_id  max_fare  min_fare  avg_fare  trip_duration
0   7b0a1     75.05     10.25     22.46             30
1   a865d    333.33      3.85     11.23             15
2   f3d2a    105.25      0.02     11.34             15
3   a7b6f    111.11      0.02     11.67             15
4   20c7f    145.25      6.45     30.46             30
5   078cc    103.75      1.00     10.76             15
6   0dea8     83.75      1.00     11.27             15
7   8c8c7     68.25      4.00     11.00             15
8   e61b0     30.45      3.05     10.47             15
9   94432     79.65      5.85     27.28             30


#### Q4: Which drop-off areas have the highest average tip?

In [67]:
highest_tip = """

SELECT
dropoff_community_area,
ROUND(AVG(tips),2) AS avg_tips,
MAX(tips) AS maximum_tip_for_area
FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
WHERE
dropoff_community_area IS NOT NULL
GROUP BY dropoff_community_area
ORDER BY avg_tips DESC
LIMIT 15


"""

In [68]:
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(highest_tip, job_config=safe_config)
tip_result = query_job.to_dataframe()
print(tip_result)

    dropoff_community_area  avg_tips  maximum_tip_for_area
0                       76      4.04                596.85
1                       56      3.40                999.99
2                       72      3.08                 80.00
3                       74      2.28                150.00
4                       41      2.06                250.00
5                        9      2.05                 74.75
6                       64      1.91                100.00
7                       12      1.58                220.00
8                        5      1.49                486.00
9                       39      1.47                212.50
10                      33      1.39                346.00
11                      10      1.36                255.89
12                       4      1.33                900.00
13                       6      1.31                666.66
14                      57      1.30                 65.00


#### Q5: How does trip duration affect fare rates for trips lasting less than 90 minutes?